# Get_Template: 
This script constructs a very simple `template` of a miyake event. The sinusoidal component of the production function is ignored resulting in a smooth curve. The template is constructed using the best fitting `model`-`dataset` pair  from the 774AD year. `emcee` is used to fit the parameters of the production function which is then used to generate the `.csv` file containing the `dc14` concentrations over just the event.

In [121]:
from os import getcwd
from numpy import exp, array, append, vectorize, mean, transpose
from numpy import float32
from scipy.optimize import minimize
from scipy.integrate import odeint
from plotnine import ggplot, aes, geom_point


In [130]:
YEAR = 0
DC14 = 1
SIG_DC14 = 2

In [131]:
with open(f"{getcwd()}/datasets/775AD/NH/Miyake12_Cedar.csv") as data:
    _ = next(data)
    data = array([row.strip().split(",") for row in data], dtype=float32)
    data = transpose(data)

So for just modelling the troposphere I will want to set things up so that I have a differential equation where $f(x) = e^{x^{8}}$ is the rate of change. I will then use `scipy.optimize.minimize()` to minimize a loss function with respect to the solution of the ode as determined by the `return` of `ode.odeint`. 

In [64]:
def production(t, *args):
    peak, span, height = args  # Loading parameters
    return height * exp(-((t - peak) / (span / 2)) ** 8)   # Gaussian pulse

In [40]:
def loss(params=()):
    ode.odeint(dydx, mean(data["dc14"][:4]), data["year"], *params,)
    


In [138]:
odeint(dydx, mean(data[DC14][:4]), data[YEAR], args=(775, 1))



array([[-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076],
       [-17.45000076]])

`pandas.read_csv`: 1.02 ms ± 65.6 µs

`list` with `append`: 57.5 µs ± 3.6 µs

`ndarray` with `append`: 668 µs ± 60.6 µs 

`ndarray` with `map`, `numpy.append`: 38.2 µs ± 4.07 µs

`ndarray` with `map`, `add_row`: 17.7 µs ± 1.34 µs 

`list` with `numpy.array` conversion at the end: 53.7 µs ± 3.14 µs